# Fourier Part 1: Option pricing with Fourier transformation

<font color ="#4169e1"> Using Fourier tranform to price a Vanilla call option </font>

## <font color="#caa520">Basics</font>

### Fourier transform

Suppose f(x) is a function such as:

$$ \int_{\mathbb{R}^n	} | f(x)| dx < \infty  $$

The Fourier transform of this function can be written as:
$$ f(\omega) := \int_{\mathbb{R}^n	}  f(x) e^{i{\omega}x} dx  $$ 

The $ \mathbb{R}^n$ in the above Fourier transform equation can be replaced with $-\infty $ and $\infty $ if we limit ourselves to one-dimensional cases.

$$ f_X(\omega) := \int_{-\infty}^{\infty}  f(x) e^{i{\omega}x} dx  $$ 

### Inverse Fourier transform

Given integribility and continuity, we can get back the original function from the Fourier transform as follows:

$$ f(x) = \frac{1}{2 \pi} \int_{-\infty}^{\infty}  f(\omega) e^{-i{\omega}x} d\omega  $$ 

## <font color="#caa520">Characterstic functions</font>

If X has a pdf (probability density function), then its characterstic function is given by the Fourier transform of the pdf. If two random variables have the same characterstic functions, then they have the same distribution.

$$ \varphi_{X}(t) := \mathbb{E}[e^{itX}] $$

If $f_X(x) $ is the pdf, we can perform a Fourier transform on this pdf as follows:

$$ f_X(\omega) := \int_{-\infty}^{\infty}  f(x) e^{i{\omega}x} dx  $$ 

Hence, $$ f_X(\omega) := \mathbb{E}[e^{i{\omega}X}]  $$ 

$$ f_X(\omega) := \varphi_{X}(\omega)  $$

Also, we can go back to the pdf from the characterstic function using inverse Fourier transform, assumption integribility and continuity conditions are met

$$ f(x) = \frac{1}{2 \pi} \int_{-\infty}^{\infty}  \varphi_{X}(\omega)  e^{-i{\omega}x} d\omega  $$ 

### Calculating probabilities from characterstic functions

We use the following theorem:

$$ \mathbb{P}[X<= x] = \frac{1}{2} - \frac{1}{\pi} \int_{0}^{\infty} \frac{Im[e^{itX}{\varphi}_X(t)]}{t} dt $$

## <font color="#caa520">Call pricing using Fourier </font>

Without going into the derivation, we can say that the price of a call option under Fourier transforms can be given by the following formula 

$$ c = S_0(\frac{1}{2} + \frac{1}{\pi} \int_0^{\infty}\frac{Im[e^{-it \ln K}\varphi_{M_2}(t)]}{t}dt) -e^{-rT}K(\frac{1}{2} + \frac{1}{\pi} \int_0^{\infty}\frac{Im[e^{-it \ln K}\varphi_{M_1}(t)]}{t}dt) $$

where

$$ \varphi_{M_1}(t) = e^{(it(\ln S_0 + (r-\frac{\sigma^2}{2})T)-\frac{1}{2} \sigma^2 T t^2)}$$
$$\varphi_{M_2}(t) = e^{it \sigma^2 T} \varphi_{M_1}(t) $$

We have two integrals in the above equation. To evaluate them, we will break them down using the following method

$$\int_0^{\infty}\frac{Im[e^{-it \ln K}\varphi_{M_2}(t)]}{t}dt \approx \int_0^{t_{max}}\frac{Im[e^{-it \ln K}\varphi_{M_2}(t)]}{t}dt \approx \Sigma_{n=1}^N \frac{Im[e^{-it_n \ln K}\varphi_{M_2}(t)]}{t_n} \Delta t$$

We need to chose $t_{max}$ and N to find $\Delta t = \frac{t_{max}}{N}$

## <font color="#caa520">Application in Python </font>

### a) Analytical pricing using Black-Scholes

In [20]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt

# share 
r = 0.02
S0 = 100
sigma = 0.20

# call option 
K = 110
T = 1
k_log = np.log(K)

In [21]:
#Analytical pricing using the Black-Scholes option pricing model
d1_stock = (np.log(S0/K) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
d2_stock = d1_stock - sigma*np.sqrt(T)

analytic_callprice = S0*norm.cdf(d1_stock) - K*np.exp(-r*T)*norm.cdf(d2_stock) 
analytic_callprice

4.94386695723049

### b) Fourier pricing

In [22]:
# Characteristic functions where 1j is used to create the imaginary number i
def c_M1(t):
    return np.exp(1j*t*(np.log(S0)+(r-sigma**2/2)*T) - (sigma**2)*T*(t**2)/2)

def c_M2(t):
    return np.exp(1j*t*sigma**2*T)*c_M1(t)

In [23]:
# Choosing t_max and N
t_max = 20
N = 100

# Calculating delta and constructing t_n
delta_t = t_max/N
from_1_to_N = np.linspace(1, N, N)
t_n = (from_1_to_N - 1/2)*delta_t

In [24]:
# Approximate integral estimates
first_integral = sum((((np.exp(-1j*t_n*k_log)*c_M2(t_n)).imag)/t_n)*delta_t)
second_integral = sum((((np.exp(-1j*t_n*k_log)*c_M1(t_n)).imag)/t_n)*delta_t)

In [25]:
# Fourier Call Value
fourier_call_val = S0*(1/2 + first_integral/np.pi) - np.exp(-r*T)*K*(1/2 + second_integral/np.pi)
fourier_call_val

4.943834589108832

We can see that we get the same price for the call option using both methods.

## <font color="#caa520">References </font>

* Yves Hilpisch, "Chapter 6: Derivatives Analytics with Python: Data Analysis, Models, Simulation, Calibration and Hedging"
* Fourier methods, Python codes: https://github.com/yhilpisch/dawp/tree/master/python36/06_fou
* WQU notes, Computational Finance